<a href="https://colab.research.google.com/github/neuralsrg/SupervisedLearning/blob/main/Tensorflow/%5Bhand_signs%5Dtf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating the same Neural Net (as in tf_manual_nn) using tf functions

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset

In [1]:
import h5py

train_dataset = h5py.File('/content/drive/MyDrive/HandSigns/train_signs.h5', "r")
test_dataset = h5py.File('/content/drive/MyDrive/HandSigns/test_signs.h5', "r")

In [5]:
np.array(train_dataset['train_set_x']).shape

(1080, 64, 64, 3)

In [6]:
X_train_old = tf.constant(train_dataset['train_set_x'])#.reshape(-1, 64 * 64 * 3) / 255.
X_test_old = tf.constant(test_dataset['test_set_x'])# .reshape(-1, 64 * 64 * 3) / 255.
y_train_old = tf.constant(train_dataset['train_set_y'])
y_test_old = tf.constant(test_dataset['test_set_y'])

X_train_old = tf.reshape(tf.cast(X_train_old, dtype=tf.float32), shape=(-1, 64*64*3))
X_train_old = X_train_old / 255.

X_test_old = tf.reshape(tf.cast(X_test_old, dtype=tf.float32), shape=(-1, 64*64*3))
X_test_old = X_test_old / 255.

X_train_old = tf.expand_dims(X_train_old, axis=1)
X_test_old = tf.expand_dims(X_test_old, axis=1)

#X_train_old = X_train_old.reshape(-1, 1, 64*64*3)
#X_test_old = X_test_old.reshape(-1, 1, 64*64*3)

# X_train_old = X_train_old[0].reshape(1, 1, -1)
# y_train_old = np.array([y_train_old[0]])

# X_train_old = tf.cast(tf.constant(X_train_old), dtype=tf.float32)
# y_train_old = tf.constant(y_train_old)

In [7]:
X_train_old.shape

TensorShape([1080, 1, 12288])

## New Dataset

In [11]:
train_dataset = h5py.File('/content/drive/MyDrive/HandSigns/train_signs.h5', "r")
test_dataset = h5py.File('/content/drive/MyDrive/HandSigns/test_signs.h5', "r")

x_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])

x_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

def normalize(image):
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.reshape(image, [-1, ])
  
  return image

norm_train = x_train.map(normalize)
norm_test = x_test.map(normalize)

def one_hot(label, depth=6):
  return tf.reshape(tf.one_hot(label, depth, axis=0), [depth])

norm_y_train = y_train.map(one_hot)
norm_y_test = y_test.map(one_hot)

# Dataset

X_train = tf.constant(list(norm_train.as_numpy_iterator()))
X_train = tf.expand_dims(X_train, axis=1)

X_test = tf.constant(list(norm_test.as_numpy_iterator()))
X_test = tf.expand_dims(X_test, axis=1)

y_train = tf.constant(list(norm_y_train.as_numpy_iterator()))
y_test = tf.constant(list(norm_y_test.as_numpy_iterator()))

## Model

In [8]:
tf.random.set_seed(32)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(25,
                                # activation='relu',
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(12,
                                # activation='relu',
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(6,
                                activation='softmax',
                                kernel_initializer='glorot_uniform'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


'''
tf.random.set_seed(32)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(25,
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(12,
                                kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(6,
                                activation='softmax',
                                kernel_initializer='glorot_uniform'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
'''

"\ntf.random.set_seed(32)\n\nmodel = tf.keras.models.Sequential()\nmodel.add(tf.keras.layers.Flatten())\nmodel.add(tf.keras.layers.Dense(25,\n                                kernel_initializer='glorot_uniform'))\nmodel.add(tf.keras.layers.Dense(12,\n                                kernel_initializer='glorot_uniform'))\nmodel.add(tf.keras.layers.Dense(6,\n                                activation='softmax',\n                                kernel_initializer='glorot_uniform'))\n\nmodel.compile(\n    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),\n    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),\n    metrics=['accuracy']\n)\n"

In [9]:
history = model.fit(
    x=X_train_old,
    y=y_train_old,
    batch_size=32,
    epochs=100,
    verbose=1,
    validation_data=(X_test_old, y_test_old)
)

Epoch 1/100
34/34 [==============================] - 1s 11ms/step - loss: 2.2566 - accuracy: 0.1593 - val_loss: 1.9382 - val_accuracy: 0.2000
Epoch 2/100
34/34 [==============================] - 0s 7ms/step - loss: 1.8844 - accuracy: 0.2398 - val_loss: 1.9019 - val_accuracy: 0.2250
Epoch 3/100
34/34 [==============================] - 0s 6ms/step - loss: 1.7641 - accuracy: 0.2806 - val_loss: 1.8878 - val_accuracy: 0.2583
Epoch 4/100
34/34 [==============================] - 0s 6ms/step - loss: 1.6490 - accuracy: 0.3278 - val_loss: 1.5099 - val_accuracy: 0.4583
Epoch 5/100
34/34 [==============================] - 0s 6ms/step - loss: 1.4249 - accuracy: 0.4602 - val_loss: 1.4929 - val_accuracy: 0.4000
Epoch 6/100
34/34 [==============================] - 0s 7ms/step - loss: 1.3653 - accuracy: 0.5037 - val_loss: 1.3972 - val_accuracy: 0.4250
Epoch 7/100
34/34 [==============================] - 0s 7ms/step - loss: 1.3055 - accuracy: 0.5222 - val_loss: 1.3289 - val_accuracy: 0.4917
Epoch 8/100


KeyboardInterrupt: ignored

In [ ]:
model(X_train_old)#.reshape(-1, 1, 64*64*3))
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_26 (Flatten)        (1080, 12288)             0         
                                                                 
 dense_78 (Dense)            (1080, 25)                307225    
                                                                 
 dense_79 (Dense)            (1080, 12)                312       
                                                                 
 dense_80 (Dense)            (1080, 6)                 78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# parameters

W1, b1 = model.layers[1].get_weights()
W1 = tf.Variable(W1)
b1 = tf.Variable(b1.reshape(1, -1))

W2, b2 = model.layers[2].get_weights()
W2 = tf.Variable(W2)
b2 = tf.Variable(b2.reshape(1, -1))

W3, b3 = model.layers[3].get_weights()
W3 = tf.Variable(W3)
b3 = tf.Variable(b3.reshape(1, -1))

parameters = {
    'W1' : tf.Variable(W1),
    'b1' : tf.Variable(b1),
    'W2' : tf.Variable(W2),
    'b2' : tf.Variable(b2),
    'W3' : tf.Variable(W3),
    'b3' : tf.Variable(b3)
}

params1 = {
    'W1' : tf.Variable(W1),
    'b1' : tf.Variable(b1),
    'W2' : tf.Variable(W2),
    'b2' : tf.Variable(b2),
    'W3' : tf.Variable(W3),
    'b3' : tf.Variable(b3)
}

## Training

In [ ]:
# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 0.99**(epoch))

for i in range(50):
  W1, b1 = model.layers[1].get_weights()
  tf_W1_old = np.copy(W1)

  history = model.fit(
      x=X_train_old,
      y=y_train_old,
      batch_size=108,
      epochs=1,
      verbose=1
  )

  W1, b1 = model.layers[1].get_weights()
  tf_W1_new = np.copy(W1)

## Manual

In [ ]:
def normalize(image):
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.reshape(image, [-1, ])
  
  return image

def forward(X, parameters):

  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  W3 = parameters['W3']
  b3 = parameters['b3']

  #print(X.shape, W1.shape, b1.shape)
  Z1 = tf.math.add(tf.linalg.matmul(X, W1), b1)
  A1 = tf.keras.activations.relu(Z1)

  #print(A1.shape, W2.shape, b2.shape)
  Z2 = tf.math.add(tf.linalg.matmul(A1, W2), b2)
  A2 = tf.keras.activations.relu(Z2)

  Z3 = tf.math.add(tf.linalg.matmul(A2, W3), b3)
  A3 = tf.keras.activations.softmax(Z3)
  
  return A3

def cost(preds, labels):

  # tf.keras.categorical_crossentropy expects both preds and lables of shape 
  # (num_samples, num_classes)
  cost = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(labels, preds))

  return cost

In [ ]:
def modeling(X_train, Y_train, lr=0.0001):

  # optimizer - Adam
  optimizer = tf.keras.optimizers.SGD(lr)
  
  i = 9

  X = X_train[i * 108: (i + 1) * 108, :]
  Y = Y_train[i * 108: (i + 1) * 108]

  with tf.GradientTape() as tape:
    A3 = forward(X, parameters)
    minibatch_cost = cost(A3, Y)

  print(f'i = {i} : cost = {minibatch_cost}')

  # print(f'Minibatch cost was: {minibatch_cost}')
  trainable = parameters.values()

  # back propagation
  grads = tape.gradient(minibatch_cost, trainable)
  optimizer.apply_gradients(zip(grads, trainable))

  return parameters

In [ ]:
man_W1_old = np.copy(parameters['W1'])
parameters = modeling(tf.squeeze(X_train_old), tf.one_hot(y_train_old, depth=6))
man_W1_new = np.copy(parameters['W1'])

i = 9 : cost = 1.9321264028549194


## Comparing 

In [ ]:
tf_W1_old

array([[ 0.00199481, -0.01617088, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01953882],
       [-0.00774924, -0.01985233,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02190093],
       [-0.01986766, -0.00848458,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01421347],
       ...,
       [-0.01084893,  0.00037564, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00841468],
       [-0.01125521, -0.00640606,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609643],
       [ 0.01956861, -0.00456034,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.0011325 ]], dtype=float32)

In [ ]:
man_W1_old

array([[ 0.00199481, -0.01617088, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01953882],
       [-0.00774924, -0.01985233,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02190093],
       [-0.01986766, -0.00848458,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01421347],
       ...,
       [-0.01084893,  0.00037564, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00841468],
       [-0.01125521, -0.00640606,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609643],
       [ 0.01956861, -0.00456034,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.0011325 ]], dtype=float32)

In [ ]:
tf_W1_new

array([[ 0.00199546, -0.01616882, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01953681],
       [-0.00774861, -0.01985034,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02190286],
       [-0.01986707, -0.00848269,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01421161],
       ...,
       [-0.01084851,  0.00037761, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00841307],
       [-0.01125483, -0.00640414,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.006098  ],
       [ 0.01956898, -0.00455849,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113094]], dtype=float32)

In [ ]:
man_W1_new

array([[ 0.00199481, -0.01617111, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01953697],
       [-0.00774923, -0.01985255,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02190271],
       [-0.01986765, -0.00848482,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01421176],
       ...,
       [-0.01084906,  0.00037577, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00841326],
       [-0.01125535, -0.00640587,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609775],
       [ 0.01956847, -0.00456012,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113124]], dtype=float32)

In [ ]:
(tf_W1_new - man_W_new)[(tf_W1_new - man_W_new) != 0].max()

0.00013424084

In [ ]:
np.abs((tf_W1_old - tf_W1_new) / 0.0001 - (man_W1_old - man_W1_new) / 0.0001).max()

0.508558

### Entire batch

In [ ]:
def modeling1(X_train, Y_train, lr=0.0001):
  
  costs = 0

  # optimizer - Adam
  optimizer = tf.keras.optimizers.SGD(lr)
  
  for i in range(10):
  #i = 2

    X = X_train[i * 108: (i + 1) * 108, :]
    Y = Y_train[i * 108: (i + 1) * 108]

    with tf.GradientTape() as tape:
      A3 = forward(X, params1)
      minibatch_cost = cost(A3, Y)

    # print(f'i = {i} : cost = {minibatch_cost}')
    costs += minibatch_cost
    # print(f'Minibatch cost was: {minibatch_cost}')
    trainable = params1.values()

    # back propagation
    grads = tape.gradient(minibatch_cost, trainable)
    optimizer.apply_gradients(zip(grads, trainable))

  costs /= 10
  # print(f'Cost : {costs}')

  return params1

In [ ]:
for i in range(50):

  man_W_old = np.copy(params1['W1'])
  params1 = modeling1(tf.squeeze(X_train_old), tf.one_hot(y_train_old, depth=6))
  man_W_new = np.copy(params1['W1'])

In [ ]:
man_W_old

array([[ 0.00199581, -0.01617085, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01954625],
       [-0.00774827, -0.01985236,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02189378],
       [-0.01986675, -0.00848466,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.0142203 ],
       ...,
       [-0.01084797,  0.00037441, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00842013],
       [-0.01125425, -0.00640712,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609114],
       [ 0.01956955, -0.00456122,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113762]], dtype=float32)

In [ ]:
man_W_new

array([[ 0.00199481, -0.01617111, -0.00210474, ...,  0.00400415,
        -0.00541233,  0.01953697],
       [-0.00774923, -0.01985255,  0.00654225, ...,  0.01703717,
         0.00904857, -0.02190271],
       [-0.01986765, -0.00848482,  0.00357587, ..., -0.01890617,
        -0.0158468 ,  0.01421176],
       ...,
       [-0.01084906,  0.00037577, -0.02203674, ..., -0.00582635,
        -0.01651511,  0.00841326],
       [-0.01125535, -0.00640587,  0.00845467, ..., -0.0018795 ,
        -0.01937845, -0.00609775],
       [ 0.01956847, -0.00456012,  0.00275667, ..., -0.00196295,
         0.0200613 ,  0.00113124]], dtype=float32)

In [ ]:
for i in range(50):

  history = model.fit(
      x=X_train_old,
      y=y_train_old,
      batch_size=108,
      epochs=1,
      verbose=0
  )

  W1, b1 = model.layers[1].get_weights()
  tf_W1_new = np.copy(W1)

  params1 = modeling1(tf.squeeze(X_train_old), tf.one_hot(y_train_old, depth=6))
  man_W_new = np.copy(params1['W1'])

  print(f'epoch {i + 1}: {(tf_W1_new - man_W_new)[(tf_W1_new - man_W_new) != 0].max()}')

epoch 1: 2.518296241760254e-06
epoch 2: 4.0084123611450195e-06
epoch 3: 4.759058356285095e-06
epoch 4: 5.817040801048279e-06
epoch 5: 6.089452654123306e-06
epoch 6: 7.158145308494568e-06
epoch 7: 7.4356794357299805e-06
epoch 8: 7.599592208862305e-06
epoch 9: 7.377937436103821e-06
epoch 10: 8.473172783851624e-06
epoch 11: 7.778406143188477e-06
epoch 12: 8.005648851394653e-06
epoch 13: 1.046992838382721e-05
epoch 14: 9.931623935699463e-06
epoch 15: 9.642913937568665e-06
epoch 16: 9.859912097454071e-06
epoch 17: 1.0007992386817932e-05
epoch 18: 1.0370276868343353e-05
epoch 19: 1.2023374438285828e-05
epoch 20: 1.3644807040691376e-05
epoch 21: 1.5050172805786133e-05
epoch 22: 1.6191042959690094e-05
epoch 23: 1.7605721950531006e-05
epoch 24: 1.9249506294727325e-05
epoch 25: 2.0483508706092834e-05
epoch 26: 2.1696090698242188e-05
epoch 27: 2.4549663066864014e-05
epoch 28: 2.7369707822799683e-05
epoch 29: 3.0489638447761536e-05
epoch 30: 3.325752913951874e-05
epoch 31: 3.5159289836883545e-05
e

## Saving model

The SavedModel format (default)

In [ ]:
model.save('/content/drive/MyDrive/HandSigns/SavedModel_format')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/HandSigns/SavedModel_format/assets


In [ ]:
!ls /content/drive/MyDrive/HandSigns/SavedModel_format

assets	keras_metadata.pb  saved_model.pb  variables


The **HDF5** format

In [ ]:
model.save('/content/drive/MyDrive/HandSigns/saved_model.h5')

In [ ]:
!ls /content/drive/MyDrive/HandSigns/

SavedModel_format  saved_model.h5  test_signs.h5  train_signs.h5


## Loading model

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/HandSigns/SavedModel_format")
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 25)                307225    
                                                                 
 dense_4 (Dense)             (None, 12)                312       
                                                                 
 dense_5 (Dense)             (None, 6)                 78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/HandSigns/saved_model.h5")
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 25)                307225    
                                                                 
 dense_4 (Dense)             (None, 12)                312       
                                                                 
 dense_5 (Dense)             (None, 6)                 78        
                                                                 
Total params: 307,615
Trainable params: 307,615
Non-trainable params: 0
_________________________________________________________________
